In [5]:
#Imports
import sys
import librosa
import numpy as np
import json
from pyloudnorm import Meter

In [26]:
def generate_visualization_data_to_json(audio_path, output_path, reference_lufs=-14):
    y, sr = librosa.load(audio_path, sr=None, mono=True)
    
    # Calculate LUFS
    meter = Meter(sr)  # create meter
    lufs = meter.integrated_loudness(y)

    # Apply gain adjustment
    gain_adjustment = 10 ** ((reference_lufs - lufs) / 20)
    y = y * gain_adjustment

    n_fft = 2048
    hop_length = 512

    peak_frequencies = [63, 160, 400, 1000, 2500, 6250]

    spectrogram = np.abs(librosa.stft(y, n_fft=n_fft, hop_length=hop_length))
    freqs = librosa.fft_frequencies(sr=sr, n_fft=n_fft)

    # Determine the max value across the entire audio for normalization
    max_val = np.max(spectrogram)

    # Prepare visualization data array
    time_series_band_data = []

    # Iterate over each time slice
    for time_index in range(spectrogram.shape[1]):
        moment_band_data = []
        for peak_freq in peak_frequencies:
            index = np.argmin(np.abs(freqs - peak_freq))
            # Normalize each band value based on the max value across the entire file
            normalized_value = (spectrogram[index, time_index] / max_val) * 100
            moment_band_data.append(normalized_value)
        
        time_series_band_data.append(moment_band_data)

    visualization_data = {"visualization_data": time_series_band_data}
        
    # Save as JSON
    with open(output_path, 'w') as json_file:
        json.dump(visualization_data, json_file)
        
    return np.array(time_series_band_data)


In [31]:
song_filepath = "./songs/droeloe_panorama.wav"
output_filepath = "./data/droeloe_panorama_data_lufs_standardization.json"
song_data = generate_visualization_data_to_json(song_filepath, output_filepath)

In [28]:
song_filepath = "./songs/droeloe_sunburn.wav"
output_filepath = "./data/droeloe_sunburn_data_lufs_standardization.json"
song_data = generate_visualization_data_to_json(song_filepath, output_filepath)

In [29]:
song_filepath = "./songs/DROELOE_Statues.wav"
output_filepath = "./data/DROELOE_Statues_data_lufs_standardization.json"
song_data = generate_visualization_data_to_json(song_filepath, output_filepath)

In [30]:
song_filepath = "./songs/MELVV_Blank.wav"
output_filepath = "./data/MELVV_Blank_data_lufs_standardization.json"
song_data = generate_visualization_data_to_json(song_filepath, output_filepath)

In [32]:
print(song_data)
print(song_data.shape)

[[1.88728995e-04 1.28764248e-04 2.42171808e-05 7.11419190e-06
  4.51490564e-05 3.79775216e-06]
 [1.13525812e-04 6.83648125e-05 2.95947672e-05 4.81043287e-05
  9.83568953e-05 2.43780676e-05]
 [5.66025165e-05 1.64111384e-05 5.10257053e-05 6.92739150e-05
  4.01673816e-05 3.96300607e-05]
 ...
 [6.64125546e-03 7.42883049e-04 2.90565713e-03 7.87374483e-04
  1.10093617e-04 2.73190750e-06]
 [1.12789098e-04 1.79678966e-04 4.04220231e-05 6.29992030e-05
  8.53835758e-06 2.47837733e-05]
 [9.34747902e-05 2.01845478e-04 3.97102184e-05 5.97210260e-05
  5.09949523e-06 1.01756250e-05]]
(19785, 6)


In [33]:
print(song_data[10000])
print(song_data.min())
print(song_data.max())

[2.39363406 3.43709439 1.96161717 0.398194   0.02805529 0.05049458]
1.0561069352377217e-06
100.0
